# Calculations of "Meaningful Information"
**by Tom English**

See my post "<a href="http://theskepticalzone.com/wp/evo-info-5-meaningless-meaning/">Evo-Info 5: Meaningless Meaning</a>" at <em>The Skeptical Zone.</em>

In [1]:
!TZ=America/Chicago date

Fri Oct 18 09:59:33 CDT 2019


## Generation of images

In [2]:
import numpy as np
import io
import imageio
from IPython.display import Image, display

**Definitions of functions used to load, crop, and store images**

In [3]:
def fetch(path, in_wikipedia=True):
    """
    Returns an image loaded from location specified by `path`.
    
    The path can be a URL. If `in_wikipedia` is true, the given path is
    appended to the URL of the Wikipedia part of the Wikimedia Commons.
    """
    if in_wikipedia:
        path = 'https://upload.wikimedia.org/wikipedia/commons/' + path
    return imageio.imread(path)

    
def crop_to_same_shape(a, b):
    """
    Returns the overlapping regions of images `a` and `b`.
    
    The height (width, depth) of the returned images is the mimimum of
    the heights (widths, depths) of the given images.
    
    Any cropping that occurs is at the bottom and/or right of the image.
    If one of the images is gray-scale, and the other is color, then
    only the first of the RGB (or HSV) values of each pixel in the color
    image is preserved.
    """
    # Construct a slice that includes the regions of the images that
    # overlap when the images are aligned at the upper left-hand corner.
    dim = [min(m, n) for m, n in zip(a.shape, b.shape)]
    overlap = np.s_[:dim[0], :dim[1], :dim[2]]
    #
    # Return the overlapping parts of the images.
    return a[overlap], b[overlap]


def write_png(path_without_extension, image):
    """
    Stores `image` in PNG format, and returns the PNG file size in bits.
    
    The `path_without_extension` is extended by '.png' to obtain the
    storage location.
    """
    path = path_without_extension + '.png'
    with io.FileIO(path, 'wb') as compressed:
        imageio.imwrite(compressed, image, format='png')
        png_bits = 8 * compressed.tell()
    return png_bits

**Image processing**

1. Load an image of Mount Fuji and an image of Mount Rushmore from the Wikimedia Commons.
2. Crop the images of Fuji and Rushmore, as necessary, to make them identical in dimensions.
3. Create two new images by performing very simple operations on the images of Fuji and Rushmore. The new images are identical in dimensions to those of Fuji and Rushmore.
4. Save the four images to disk in PNG format, and report the sizes of the PNG files.

In [4]:
"""
Specify the paths to images of Mount Fuji and Mount Rushmore within the
Wikipedia part of the Wikimedia Commons.
"""
fuji_path = ("f/f7/Mount_Fuji_Japan_with_Snow%2C_Lakes_and_"
              + "Surrounding_Mountains.jpg")
rush_path = ("5/58/Aerial_view_of_Mount_Rushmore_National_Memorial_"
              + "by_Volkan_Yuksel_DSC04244.JPG")

"""
Load the Fuji and Rushmore images from the Wikimedia Commons, align them
at their upper left-hand corners, and crop them to the same dimensions.
Then report the dimensions and sizes of the images.
"""
fuji, rush = crop_to_same_shape(fetch(fuji_path), fetch(rush_path))
n_raw_bits = 8 * np.prod(fuji.shape)
print('Image width, height, and depth:', fuji.shape)
print('Number of bits per raw image  : {:,}'.format(n_raw_bits))

"""
Produce two images, one by taking the bitwise OR of Fuji and Rushmore,
and the other by taking the bitwise AND of Fuji and Rushmore. Then
produce a single image by taking the 8-bit products (discarding overflow)
of the corresponding RGB values of the OR image and the AND image.
Finally, invert every bit in the resulting image.
"""
mung = ~((fuji | rush) * (fuji & rush))

"""
Create an image that is the cumulative 8-bit sum (with overflow bits
discarded) of RGB values in the Rushmore image.
"""
csum = rush.flatten()
np.cumsum(csum, out=csum)
csum = csum.reshape(rush.shape)

"""
Write all of the images to disk in PNG format.
"""
n_rush = write_png('ET Views Rushmore', rush)
n_fuji = write_png('Fuji at Dawn', fuji)
n_mung = write_png('The Languor of Egnorance', mung)
n_csum = write_png('The Apotheosis of Meaning', csum)

"""
Report the sizes, in bits, of the PNG files.
"""
print('\nNumber of bits in each of the compressed images')
print('   Fuji at Dawn             : {:11,}'.format(n_fuji))
print('   E.T. Views Rushmore      : {:11,}'.format(n_rush))
print('   The Languor of Egnorance : {:11,}'.format(n_mung))
print('   The Apotheosis of Meaning: {:11,}'.format(n_csum))

Image width, height, and depth: (2592, 3456, 3)
Number of bits per raw image  : 214,990,848

Number of bits in each of the compressed images
   Fuji at Dawn             :  69,143,232
   E.T. Views Rushmore      : 102,385,888
   The Languor of Egnorance : 193,402,368
   The Apotheosis of Meaning: 206,639,528


**Display of images**

The following images are scaled down to fit within the cell. To view an image at full resolution (at least in some browsers), click on its title.

[*Fuji at Dawn*](./Fuji%20at%20Dawn.png)
<img src="./Fuji%20at%20Dawn.png">

&nbsp;

[*E.T. Views Rushmore*](./ET%20Views%20Rushmore.png)
<img src="./ET%20Views%20Rushmore.png">

&nbsp;

[*The Languor of Egnorance*](./The%20Languor%20of%20Egnorance.png)
<img src="./The%20Languor%20of%20Egnorance.png">

&nbsp;

[*The Apotheosis of Meaning*](./The%20Apotheosis%20of%20Meaning.png)
<img src="./The%20Apotheosis%20of%20Meaning.png">

## Discussion

Ewert, Dembski, and Marks regard *algorithmic specified complexity* (ASC) as a measure of meaningful information. We shall follow their article "<a href="http://robertmarks.org/REPRINTS/2015%20Measuring%20meaningful%20information%20in%20images.pdf">Measuring Meaningful Information in Images: Algorithmic Specified Complexity</a>" (2015) in approximating the ASC of an image:

    ASC(image) = size(compressed(image)) - K(image|context) bits,

where `size(compressed(image))` is the number of bits in the image when it is losslessly compressed for storage in Portable Network Graphics (PNG) format, and `K(image|context)` is, loosely, the length of the shortest binary program that outputs `image` on input of the binary string `context`. (Note that the size of the compressed image varies somewhat from one implementation of the PNG compressor to another.) The value of `K(image|context)` indicates the size of `image` when compressed in an alternative scheme. To put the alternative scheme concretely, if Alice and Bob have agreed upon a universal computer and the `context`, then Alice can transmit to Bob a program that outputs `image` on input of `context`, and Bob can run the program with input of `context` to recover `image`. So the ASC of an image is the difference in size when it is compressed by two different approaches, one of which does not exploit the context, and the other of which does.

For each of the images above, the number of bits of code required to generate the image, on input of the Wikimedia Commons, is tiny in comparison to the number of bits in the PNG-compressed image. Thus

    ASC(image) ≈ size(compressed(image))

for each of the images. Note that the ASC of *Languor* is much greater than the sum of the ASC values for the images in the context:

In [5]:
size_sum = n_fuji + n_rush
print('PNG size of The Languor of Egnorance :', n_mung)
print('Sum of PNG sizes of Fuji and Rushmore:', size_sum)
print('Difference                           : ', n_mung - size_sum)

PNG size of The Languor of Egnorance : 193402368
Sum of PNG sizes of Fuji and Rushmore: 171529120
Difference                           :  21873248


Thus we have "created" almost 22 megabits of "meaningful information" by running a short program (much less than 22 megabits in length) that combines the given images, *Fuji* and *Rushmore*, into a new image&nbsp;&mdash; no matter that there is loss of information in the sense that neither of the input images can be recovered from the output image. The absurdity is even more pronounced in the case of *Apotheosis*, in which a very short program operates only on the context image *Rushmore*. The "meaningful information" of the output of the program is about twice as great as that of *Rushmore*. That is, according to Ewert, Dembski, and Marks, there is not loss of information, but instead a gain of over 100 million bits of meaningful information when *Rushmore* is scrambled irreversibly to produce the patently meaningless *Apotheosis of Meaning*.

Note that the images have been scaled down for display above. If we had calculated the ASC of the lower-resolution images on display, then we would have obtained much smaller values than we did for the full-resolution images.

**Image credits**

* <a href="https://commons.wikimedia.org/wiki/File:Mount_Fuji_Japan_with_Snow,_Lakes_and_Surrounding_Mountains.jpg">Fuji</a>: Kpravin2, <a href="https://creativecommons.org/licenses/by-sa/4.0/deed.en">CC BY-SA 4.0</a> and
* <a href="https://commons.wikimedia.org/wiki/File:Aerial_view_of_Mount_Rushmore_National_Memorial_by_Volkan_Yuksel_DSC04244.JPG">Rushmore</a>: <a href="https://commons.wikimedia.org/wiki/User:Volkan_Yuksel">Volkan Yuksel</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0/deed.en">CC BY-SA 3.0</a>,

both via the Wikimedia Commons.